In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import datetime
import math
import statistics
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from itertools import product
from math import sqrt
import json
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.base import TransformerMixin #gives fit_transform method for free

#Models
from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, LSTM, Dropout,Reshape
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import clone_model
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import matthews_corrcoef, balanced_accuracy_score
from sklearn.feature_selection import RFE

#Graphs
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import periodogram
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
np.random_seed=42

In [ ]:
sample=pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
train=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [ ]:
#Lets first split our train dataset into the training portions and validation portions
X = train.drop(columns=['Transported'])
y = train['Transported']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Shows number of rows in X_train
X_train.shape[0]

In [ ]:
#Next,lets have a look at our data

def data_sum(dataframe):
    nulls=[]
    count=[]
    unique=[]
    null_percentage=[]
    dtype=[]

    for col in dataframe.columns:
        nulls.append(dataframe[col].isnull().sum())
        count.append(dataframe[col].count())
        unique.append(dataframe[col].nunique())
        total_rows=dataframe.shape[0]
        null_percentage.append(dataframe[col].isnull().sum()*100/total_rows)
        dtype.append(dataframe[col].dtype)
    
    summary=pd.DataFrame({
        'Column': dataframe.columns,
        'Nulls': nulls,
        'Non-Null Count': count,
        'Unique Values': unique,
        'Null Percentage': null_percentage,
        'Data Type': dtype
    })

    return(summary)

X_train_summary=data_sum(X_train)
print(X_train_summary)

In [ ]:
#Lets first drop some colums we wont be using
X_train=X_train.drop(columns=['PassengerId','Name'])

In [ ]:
# X_train['TotalSpending'] = X_train[['FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']].sum()

In [ ]:
X_train

In [ ]:
X_train.corr()

First lets split into deck num side 

In [ ]:
X_train[["Deck","Num","Side"]]=X_train["Cabin"].str.split("/",expand = True)

In [ ]:
X_train.drop(columns = ['Cabin'], inplace = True)

In [ ]:
#Now im trying to impute using interpolation
# Age & Amenities has a relationship. The older, the more spending.
X_train[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']] = X_train[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']].interpolate(method='linear')

In [ ]:
#lets see how the dataframe is like now
X_train_2_summary=data_sum(X_train)
X_train_2_summary

Looks like we have completed the imputing for numerical features now lets work on the categorical features.



<!--  --> from sklearn.impute import SimpleImputer. Constant strategy is better for one-hot encoding


In [ ]:
categorical = X_train.dtypes == 'object'
categorical

Look at the columns with categorical data.
Starting from HomePlanet.

Aus help to put some imputation for Arden
Arden - Above
Austin - Below

In [ ]:
#Created a new feature
X_train["TotalSpending"]=X_train[["RoomService","Spa","FoodCourt","VRDeck",'ShoppingMall']].sum(axis=1)

Lets impute HomePlanet

In [ ]:
#For rows where ["Destination"]=="PSO J318.5-22",let the rows with missing values in ["HomePlanet"] be imputed with "Earth"
X_train.loc[X_train["Destination"]=="PSO J318.5-22","HomePlanet"] = X_train.loc[X_train["Destination"]=="PSO J318.5-22","HomePlanet"].fillna("Earth")
#For rows where ["Destination"]=="55 Cancri e",let the rows with missing values in ["HomePlanet"] be imputed with "Europa"
X_train.loc[X_train["Destination"]=="55 Cancri e","HomePlanet"]=X_train.loc[X_train["Destination"]=="55 Cancri e","HomePlanet"].fillna("Europa")
#For rows where ["Destination"]=="TRAPPIST-1e",let the rows with missing values in ["HomePlanet"] be imputed with "Earth"
X_train.loc[X_train["Destination"]=="TRAPPIST-1e","HomePlanet"]=X_train.loc[X_train["Destination"]=="TRAPPIST-1e","HomePlanet"].fillna("Earth")
X_train.loc[X_train['Deck']=='A','HomePlanet']=X_train.loc[X_train['Deck']=='A','HomePlanet'].fillna("Europa")
X_train.loc[X_train['Deck']=='B','HomePlanet']=X_train.loc[X_train['Deck']=='B','HomePlanet'].fillna("Europa")
X_train.loc[X_train['Deck']=='C','HomePlanet']=X_train.loc[X_train['Deck']=='C','HomePlanet'].fillna("Europa")
X_train.loc[X_train['Deck']=='T','HomePlanet']=X_train.loc[X_train['Deck']=='T','HomePlanet'].fillna("Europa")
X_train.loc[X_train['Deck']=='G','HomePlanet']=X_train.loc[X_train['Deck']=='G','HomePlanet'].fillna("Earth")
X_train.loc[X_train['Destination'] == 'TRAPPIST-1e', 'HomePlanet'] = X_train.loc[X_train['Destination'] == 'TRAPPIST-1e', 'HomePlanet'].fillna('Earth')
X_train['HomePlanet']=X_train['HomePlanet'].fillna('Earth')

Lets impute CryoSleep

In [ ]:
X_train.loc[X_train["TotalSpending"]>=5000,"CryoSleep"]=X_train.loc[X_train["TotalSpending"]>=5000,"CryoSleep"].fillna(False)
X_train.loc[X_train["TotalSpending"]==0,"CryoSleep"]=X_train.loc[X_train["TotalSpending"]==0,"CryoSleep"].fillna(True)
X_train["CryoSleep"]=X_train["CryoSleep"].fillna(False)

Lets impute Destination

In [ ]:
X_train.loc[X_train['VIP'] == 'FALSE','Destination'] = X_train.loc[X_train['VIP'] == 'FALSE','Destination'].fillna('TRAPPIST-1e')
X_train.loc[X_train['HomePlanet'] == 'Earth', 'Destination'] = X_train.loc[X_train['HomePlanet'] == 'Earth', 'Destination'].fillna(method = 'ffill')
X_train.loc[X_train['HomePlanet'] == 'Earth', 'Destination'] = X_train.loc[X_train['HomePlanet'] == 'Earth', 'Destination'].fillna(method = 'bfill')
X_train.loc[X_train['HomePlanet'] == 'Europa', 'Destination'] = X_train.loc[X_train['HomePlanet'] == 'Europa', 'Destination'].fillna('55 Cancri e')
X_train.loc[X_train['HomePlanet'] == 'Mars', 'Destination'] = X_train.loc[X_train['HomePlanet'] == 'Mars', 'Destination'].fillna('TRAPPIST-1e')
X_train.loc[X_train['HomePlanet'] == 'Earth','Destination'] = X_train.loc[X_train['HomePlanet'] == 'Earth','Destination'].fillna('TRAPPIST-1e')
X_train.loc[X_train['VIP'] == False, 'Destination'] = X_train.loc[X_train['VIP'] == False, 'Destination'].fillna('TRAPPIST-1e')
X_train.loc[X_train['HomePlanet'] == 'Earth', 'Destination'] = X_train.loc[X_train['HomePlanet'] == 'Earth', 'Destination'].fillna('TRAPPIST-1e')

Lets impute VIP

In [ ]:
X_train.loc[X_train['TotalSpending'] >= 5000, 'VIP'] = X_train.loc[X_train['TotalSpending'] >= 5000, 'VIP'].fillna(True)
X_train.loc[X_train['TotalSpending'] < 5000, 'VIP'] = X_train.loc[X_train['TotalSpending'] < 5000, 'VIP'].fillna(False)

Lets impute Deck

In [ ]:
#Deck has a uneven distribution with deck G and F having a far larger value then the other decks. Lets just impute all nulls  to be F
X_train['Deck']=X_train['Deck'].fillna('F')

Lets  change datatype then impute Num

In [ ]:
#Convert ['Num'] to numerical feature
X_train['Num'] = X_train['Num'].str.replace(' ', '').astype(float)
print(f"There are initially {X_train['Num'].isna().sum()} nulls")
#Num has too much unique values and we cant do simple imputation because each unique value has such a low count that setting all 72 nulls to that value will skew the results so we just randomly impute the nulls using the existing values 
num_values = X_train['Num'].dropna().values
# Shuffle non-missing values multiple times
num_shuffled = num_values.copy()  # Make a copy to shuffle multiple times
np.random.shuffle(num_shuffled)
num_shuffled=pd.Series(num_shuffled)

non_numeric_values = num_shuffled[np.isnan(num_shuffled)]
if non_numeric_values.size == 0:
    print("All values in num_shuffled are numerical.")
else:
    print(f"There are {non_numeric_values.size} non-numeric values in num_shuffled.")
num_unique_values = np.unique(num_shuffled).size
print(f"There are {num_unique_values} unique values in the num_shuffled array.")


# Fill missing values with the shuffled values
X_train['Num']=X_train['Num'].fillna(num_shuffled)
print(f"Now there are {X_train['Num'].isna().sum()} nulls")
                                  
#for some reason the number of nulls dropped to 34 but wont drop further, im gonna impute the last 34 nulls with the most frequent value for now until I can solve why
X_train['Num']=X_train['Num'].fillna(82)
print(f"Now there are {X_train['Num'].isna().sum()} nulls")
    



Lets impute side

In [ ]:
#For Side, they are almost equal and rightfully so, lets just impute the nulls equally
for value in ['S','P']:
    null_indices = X_train[X_train['Side'].isnull()].sample(71).index
    X_train.loc[null_indices, 'Side'] = value

In [ ]:
X_train_3_summary=data_sum(X_train)
X_train_3_summary

Imputation complete
<br>
Now lets encode
<br>
Cardinality:
<br>
Low: 2-5
<br>
Moderate: 6-46
<br>
High: 50 and above

HomePlanet:Label Encoding
<br>
CryoSleep:Label Encoding
<br>
Destination:Label Encoding
<br>
VIP:Label Encoding
<br>
Deck:OHE
<br>
Side:Label Encoding
<br>
Remaining numerical features:Standardization

In [ ]:

class MyLabelEncoder(TransformerMixin):
    def __init__(self, *args, **kwargs):
        self.encoder = LabelEncoder(*args, **kwargs)
    def fit(self, x, y=0):
        print(f" The shape of the fitted is {x.shape}")
        print(f" The shape of the y is {y}")
        self.encoder.fit(x)
        return self
    def transform(self, x, y=0):
        print(x.shape)
        return self.encoder.transform(x)
    def fit_transform(self, x, y=0):
        print(f" The shape of the fitted is {x.shape}")
        #x is a dataframe
        columns=x.columns
        transformed_features=[]
        for column in columns:
            print(f" The column is {column}")
            feature_column = x[column]
            print(f" Each column has a shape of {feature_column.shape}")
            self.encoder.fit(feature_column)
            new_feature_column=self.encoder.transform(feature_column)
            transformed_features.append(new_feature_column)
        transformed_features=np.transpose(transformed_features)
        return transformed_features
        
        
            
            
            
            
        
            
        

In [ ]:
colours=np.array(['B','A','C','D','E','A','C','B','A','C','D']).reshape(-1, 1)
c_e=Pipeline(steps=[
    ('encoder',OneHotEncoder(handle_unknown='ignore'))  
])
e_c=c_e.fit_transform(colours)
e_c_df = pd.DataFrame(e_c.toarray())
e_c_df


In [ ]:
l_c_c_f=["HomePlanet", "CryoSleep", "Destination", "VIP", "Side"]
m_c_c_f=['Deck']
n_f=X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()



# Create transformers for categorical features
l_c_categorical_transformer = Pipeline(steps=[
    ('encoder', MyLabelEncoder())  
])

m_c_categorical_transformer = Pipeline(steps=[
    ('encoder',OneHotEncoder(handle_unknown='ignore'))  
])


# Create transformers for numerical features
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])


# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, n_f),
        ('m_c_cat', m_c_categorical_transformer, m_c_c_f),
        ('l_c_cat', l_c_categorical_transformer, l_c_c_f)
    ])

# Create the preprocessing pipeline
preprocessing_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Apply the preprocessing pipeline to your data
encoded_data = preprocessing_pipeline.fit_transform(X_train)

columns=np.concatenate((n_f,['A','B','C','D','E','F','G','H'] , l_c_c_f))
# Convert the transformed data back to a DataFrame
X_train_encoded = pd.DataFrame(encoded_data,columns=columns)


In [ ]:
X_train_encoded

Fantastic we are now done with encoding so lets move on to polynomial features, thing is we dont know how many degrees is the most optimal so lets just create multiple sets of different degrees and see which degree performs the best

In [ ]:
max_degree=6#Any higher and the notebook cannot withstand it as the RAM is only 30gb
polynomials={}
for degree in range(1,max_degree+1):
    poly = PolynomialFeatures(degree=degree)
    poly_features=poly.fit_transform(X_train_encoded)
    polynomials[degree+1]=poly_features
    print('Done')

In [ ]:
#Please impute X_valid the same way we imputed X_train

Lets start choosing our models, I think random forest is one good option to try out

In [ ]:

optimal_num=[]
best_scores=[]

for polynomial in polynomials.items():
    max_num_of_features=polynomial.shape[1]
    num_of_features=polynomial.shape[1]-1
    rf_classifier = RandomForestClassifier(random_state=42)
    mccs=[]
    
    while num_of_features>1:
        rfe = RFE(estimator=rf_classifier, n_features_to_select=num_of_features)
        rfe.fit(polynomial, y_train)
        selected_features = rfe.support_
        new_X_train=polynomial[selected_features]
        new_X_valid=X_valid[selected_features]
        rf_classifier.fit(new_X_train,y_train)
        y_pred=rf_classifier.predict(new_X_valid)
        mcc = matthews_corrcoef(y_train, y_pred)
        mccs.append(mcc)
        num_of_features-=1
        
    max_index = mccs.index(max(mccs))
    best_number_of_features=polynomial.shape[1]-1-max_index
    optimal_num.append(best_number_of_features)
    best_scores.append(max(mccs))
    
        
        
    



#Now lets find the best combination of polynomial degree and number of features 
print(f"Best polynomial degree is {best_scores.index(best_scores(mccs))+1}")
print(f"The number of features are {optimal_num[best_scores.index(best_scores(mccs))]}")


In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], # Regularization parameter 
              'penalty': ['l1', 'l2'], # Regularization type 
              'solver': ['liblinear', 'saga']} # Algorithm to use for optimization

In [ ]:
logreg = LogisticRegression()
grid_search = GridSearchCV(logreg, param_grid, cv=5, scoring='accuracy', n_jobs=-1) 
grid_search.fit(X_train_encoded, y_train)

In [ ]:
best_params = grid_search.best_params_ 
best_model = grid_search.best_estimator_
best_model